### Glossário do Projeto Delivery_Unit_Economics

Este glossário define os termos de negócio e os componentes técnicos usados no pipeline de ETL (Extract, Transform, Load) no Databricks.


#### 1. Termos de Negócio e Métricas Financeiras
Estes termos são cruciais para a análise de Profit & Loss (P&L) e Unit Economics.

| Termo | Definição |
| :--- | :--- |
| Marketplace Two-Sided | Modelo de negócio da plataforma, que conecta dois grupos distintos e interdependentes (Lojas/Fornecedores e Motoristas/Logística) aos Clientes. |
| Unit Economics (UE) | Análise da rentabilidade de uma única transação (o pedido). O objetivo final do pipeline é calcular o Lucro Bruto Unitário para determinar se o pedido é financeiramente viável. |
| P&L (Profit & Loss) Unitário | Análise da Demonstração de Resultados (Receitas e Custos) focada no nível do pedido. Utilizamos esta análise para determinar a Margem de Contribuição da plataforma por transação. |
| GMV (Gross Merchandise Value) | Valor Bruto da Mercadoria. A receita total gerada por um pedido, incluindo o subtotal do produto e a taxa de entrega (subtotal_bruto + delivery_fee_cliente). |
| COGS (Cost of Goods Sold) | Custo dos Produtos Vendidos. No nosso contexto, refere-se aos Custos Variáveis diretos da transação: o Custo Logístico (Repasse ao Motorista) e o Custo Transacional (Taxa de pagamento). |
| Lucro Bruto Unitário | A métrica final do pipeline. Representa a Margem de Contribuição da plataforma em cada pedido após subtrair os COGS da Receita Líquida. |


#### 2. Termos de Engenharia de Dados e Arquitetura

Estes termos descrevem como o pipeline de dados está estruturado no Databricks/Delta Lake.

| Termo | Definição |
| :--- | :--- |
| Arquitetura de Medalhões | Modelo de pipeline de dados (Bronze → Silver → Gold) usado para garantir qualidade, governança e confiabilidade dos dados à medida que eles progridem e são transformados. |


#### 2.1. Camadas do Pipeline (Medallion Architecture)

| Camada | Função Principal | Processo Chave |
| :--- | :--- | :--- |
| Bronze | Ingestão/Landing Zone. Contém dados brutos e imutáveis da fonte (CSV/Volume), após mínimas padronizações de schema e nome de coluna. | COPY INTO e Padronização de chaves (order_id). |
| Silver | Transformação/Limpeza. Onde a lógica de Unit Economics (UE) é aplicada. Dados são limpos, filtrados (delivery_status = 'DELIVERED') e as tabelas de fatos são unificadas via INNER JOIN. | Cálculo de UE (P&L Unitário). |
| Gold | Consumo/BI. Camada final com dados altamente estruturados e modelados em Star Schema para análise otimizada em ferramentas como o Power BI. | Modelagem Dimensional. |


#### 2.2. Modelagem Dimensional e Chaves

| Termo | Definição no Projeto | Contexto |
| :--- | :--- | :--- |
| Star Schema | Modelo de dados que usa uma Tabela Fato Central (tbl_fato_delivery_gold) ligada a várias Tabelas Dimensão (tbl_dim_store_gold, tbl_dim_time_gold) por chaves. Essencial para performance em BI. | Otimização de BI |
| Chave de Tempo (Order Date Key) | Coluna do tipo DATE na Tabela Fato que se liga à Dimensão Tempo. Foi criada usando a conversão explícita TO_TIMESTAMP('M/d/yyyy h:mm:ss a') para lidar com a string complexa de data/hora da fonte. | Conversão de Tipo |


